In [1]:
import random
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# --- Import DEAP ---
from deap import base, creator, tools

In [3]:
df=pd.read_csv("/Users/lubhanamutha/Desktop/Cancer-Prediction-Project/trial2/tcga_preprocessed.csv")
df.head()
df.shape

(10459, 17950)

In [5]:
df.head()

,Unnamed: 0,7SK,A1BG,A1BG_AS1,A1BG_CTD_2619J13_8,A1CF,A2M,A2ML1,A2M_AS1,A4GALT,...,hsa_mir_6080_RP13_104F24_1,hsa_mir_7162,hsa_mir_7162_AC126323_1,snoU13,snoU13_CTD_2659N19_2,snoU13_DDX19A,snoU13_KB_431C1_4,snoU13_RP11_578F21_12,binary_label,cancer_type
0,TCGA-S9-A7J2-01,-0.796991,-0.052854,0.147771,0.078559,-0.431812,0.022103,-0.054710,-0.858794,-1.129361,...,0.885146,-0.266888,0.008416,0.366884,-0.596754,1.018410,0.543622,-0.625708,1,TCGA-LGG
1,TCGA-G3-A3CH-11,0.448863,4.882204,4.685445,5.095249,3.635087,1.382139,-0.518259,1.037184,-1.533711,...,-2.443622,-0.103922,-0.266006,-1.664362,-2.028481,-1.024910,-0.696031,-0.625708,0,TCGA-LIHC
2,TCGA-EK-A2RE-01,-0.208117,-0.259900,-0.343978,-0.218950,-0.440317,-2.131612,3.594777,-1.971883,2.260710,...,0.625295,-0.273288,-0.266006,0.455931,-0.304613,0.214623,-0.977814,-0.625708,1,TCGA-CESC
3,TCGA-44-6778-01,-0.216039,-0.236235,-0.154341,-0.243607,-0.417308,1.504829,-0.499532,0.583811,-0.519009,...,0.041195,0.007575,0.121942,0.622270,-1.476780,0.533442,0.068603,-0.136845,1,TCGA-LUAD
4,TCGA-VM-A8C8-01,0.015244,-0.600260,-0.444148,-0.062025,-0.440317,0.150214,0.357702,0.051128,-1.108372,...,0.430767,-0.213768,-0.266006,0.508395,0.243015,0.403954,0.090918,1.038555,1,TCGA-LGG


In [4]:
df.info()
df.describe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10459 entries, 0 to 10458
Columns: 17950 entries, Unnamed: 0 to cancer_type
dtypes: float64(17947), int64(1), object(2)
memory usage: 1.4+ GB


<bound method NDFrame.describe of             Unnamed: 0       7SK      A1BG  A1BG_AS1  A1BG_CTD_2619J13_8  \
0      TCGA-S9-A7J2-01 -0.796991 -0.052854  0.147771            0.078559   
1      TCGA-G3-A3CH-11  0.448863  4.882204  4.685445            5.095249   
2      TCGA-EK-A2RE-01 -0.208117 -0.259900 -0.343978           -0.218950   
3      TCGA-44-6778-01 -0.216039 -0.236235 -0.154341           -0.243607   
4      TCGA-VM-A8C8-01  0.015244 -0.600260 -0.444148           -0.062025   
...                ...       ...       ...       ...                 ...   
10454  TCGA-95-7947-01 -0.240112 -0.553909 -0.595355           -0.453659   
10455  TCGA-VQ-AA6F-01 -0.796991 -0.328610 -0.553237           -0.447750   
10456  TCGA-55-6985-11  0.962167 -0.063219  0.023534            0.065953   
10457  TCGA-DD-A115-01 -0.046462  4.172279  3.856843            4.398265   
10458  TCGA-FV-A3I0-11 -0.047071  5.032143  4.821323            5.351792   

           A1CF       A2M     A2ML1   A2M_AS1    A4GA

In [6]:

# Target column
y = df['binary_label']

# Drop target column to get features
X_temp = df.drop('binary_label', axis=1)
X = X_temp.iloc[:, 1:-2]



# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

le = LabelEncoder()
# Fit on combined training and test labels
y_combined = np.concatenate((y_train, y_test))
le.fit(y_combined)

y_train = le.transform(y_train)
y_test = le.transform(y_test)

print("Dataset loaded successfully.")
print("shape of dataset",df.shape)
print("shape of x ",X.shape)
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)


Dataset loaded successfully.
shape of dataset (10459, 17950)
shape of x  (10459, 17946)
Shape of X_train: (7321, 17946)
Shape of X_test: (3138, 17946)
Shape of y_train: (7321,)
Shape of y_test: (3138,)


In [7]:
POP_SIZE = 10
N_GENERATIONS = 5
MUTATION_RATE = 0.1
N_FEATURES = X_train.shape[1]
hof = tools.HallOfFame(1)

In [8]:
import numpy as np
import random
from sklearn.linear_model import LogisticRegression  # Import Logistic Regression
from sklearn.tree import DecisionTreeClassifier      # Another light alternative
from sklearn.metrics import accuracy_score

# --- (Assuming these are defined globally) ---
# X_train, X_test, y_train, y_test = ...
# POP_SIZE = ...
# N_FEATURES = ...
# MUTATION_RATE = ...
# -------------------------------------------
#
# --- IMPORTANT ---
# For LogisticRegression to work well, your X_train and X_test
# data should be scaled first (e.g., using StandardScaler or MinMaxScaler)
# before you even start the genetic algorithm!
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)
#
# (And then convert them back to DataFrames if you're using .iloc)
# X_train = pd.DataFrame(X_train, columns=original_columns)
# X_test = pd.DataFrame(X_test, columns=original_columns)
# -----------------


def initialize_population():
    return np.random.randint(2, size=(POP_SIZE, N_FEATURES))

def fitness(individual):
    selected = [i for i in range(len(individual)) if individual[i] == 1]
    
    if len(selected) < 3:
        return 0.0,  # skip tiny subsets
    
    # Select the features based on the 'individual' (chromosome)
    X_train_sel = X_train.iloc[:, selected]
    X_test_sel = X_test.iloc[:, selected]
    
    # --- MODIFIED FOR A LIGHTER CLASSIFIER ---
    
    # Using Logistic Regression. It's very fast.
    model = LogisticRegression(
        solver='liblinear', # A good, fast solver for this
        random_state=42
    )
    try:
        model.fit(X_train_sel, y_train)
        preds = model.predict(X_test_sel)
        acc = accuracy_score(y_test, preds)
        return acc,  # Return as a tuple
    except Exception as e:
        print(f"Error in fitness evaluation with features {selected}: {e}")
        return 0.0,

def selection(population, fitnesses):
    # This is tournament selection
    idx1, idx2 = random.sample(range(len(population)), 2)
    return population[idx1] if fitnesses[idx1] > fitnesses[idx2] else population[idx2]

def crossover(parent1, parent2):
    # Standard one-point crossover
    point = random.randint(1, N_FEATURES - 1)
    child1 = np.concatenate((parent1[:point], parent2[point:]))
    child2 = np.concatenate((parent2[:point], parent1[point:]))
    return child1, child2

def mutate(individual):
    # Bit-flip mutation
    for i in range(len(individual)):
        if random.random() < MUTATION_RATE:
            individual[i] = 1 - individual[i]
    return individual


In [10]:
# --- 4. SET UP THE DEAP TOOLBOX ---
# (This was the missing part that caused your error)

# Create the fitness (maximize accuracy) and individual types
# (Weights is (1.0,) for maximizing, (-1.0,) for minimizing)
creator.create("FitnessMax", base.Fitness, weights=(1.0,)) 
creator.create("Individual", np.ndarray, fitness=creator.FitnessMax)

# Initialize the toolbox
toolbox = base.Toolbox()

# Register the "individual" generator
# This creates a single [0, 1, 0, ..., 1] array
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=N_FEATURES)

# Register the "population" generator
# This calls "individual" POP_SIZE times
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# --- Register YOUR functions ---
toolbox.register("evaluate", fitness)
toolbox.register("mate", crossover)
toolbox.register("mutate", mutate)

# Use a standard DEAP selection method (Tournament selection is common)
toolbox.register("select", tools.selTournament, tournsize=3)

# --- Define Crossover Probability ---
# (You had MUTATION_RATE defined, but not this)
CROSSOVER_RATE = 0.7


In [11]:



# --- 5. RUN THE GENETIC ALGORITHM ---

print("Starting evolution...")
population = toolbox.population(n=POP_SIZE) 
hof = tools.HallOfFame(1,similar=np.array_equal) # Create the Hall of Fame

# --- Setup Statistics ---
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("max", np.max)

logbook = tools.Logbook()
logbook.header = "gen", "avg", "max"


Starting evolution...


In [12]:

# --- Evaluate the Initial Population (Gen 0) ---
print("Evaluating initial population...")
fitnesses = map(toolbox.evaluate, population)
for ind, fit in zip(population, fitnesses):
    ind.fitness.values = fit

hof.update(population) # Update HOF with initial data

# Record and print stats for Gen 0
record = stats.compile(population)
logbook.record(gen=0, **record)
print(logbook.stream) # This prints the headers and Gen 0 stats


# --- Run the Generations ---
# (We start at gen 1 since 0 is done)
for gen in range(1, N_GENERATIONS + 1): 
    
    # 1. Select and clone
    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))

    # 2. Apply Crossover
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CROSSOVER_RATE:
            toolbox.mate(child1, child2)
            # Invalidate fitness of children after crossover
            del child1.fitness.values 
            del child2.fitness.values

    # 3. Apply Mutation
    for mutant in offspring:
        if random.random() < MUTATION_RATE:
            toolbox.mutate(mutant)
            # Invalidate fitness of mutant
            del mutant.fitness.values 

    # 4. Evaluate all individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
        
    # 5. Replace the old population and update Hall of Fame
    population[:] = offspring
    hof.update(population)
    
    # 6. Record and Print Stats for the Current Generation
    record = stats.compile(population)
    logbook.record(gen=gen, **record)
    print(logbook.stream) # Prints just the stats for the current gen


# --- After the loop, get your final answer ---
best_solution = hof[0]
print("\n--- Evolution Finished ---")
print(f"The best feature set found has an accuracy of: {best_solution.fitness.values[0]:.4f}")

# Optional: Print the details of the best solution
selected_features_indices = [i for i in range(len(best_solution)) if best_solution[i] == 1]
print(f"It selected {len(selected_features_indices)} features out of {N_FEATURES}.")
print("Selected feature indices:", selected_features_indices)

Evaluating initial population...
gen	avg     	max     
0  	0.985118	0.988209
1  	0.986839	0.988209
2  	0.98789 	0.988209
3  	0.988082	0.988209
4  	0.988209	0.988209
5  	0.988209	0.988209

--- Evolution Finished ---
The best feature set found has an accuracy of: 0.9882
It selected 9054 features out of 17946.
Selected feature indices: [6, 8, 10, 11, 12, 14, 16, 20, 25, 26, 27, 29, 31, 33, 36, 37, 38, 39, 42, 43, 45, 47, 48, 50, 52, 54, 56, 60, 62, 63, 64, 66, 67, 68, 70, 71, 72, 73, 74, 75, 76, 77, 79, 82, 85, 86, 87, 88, 89, 91, 92, 93, 94, 96, 98, 99, 100, 103, 105, 107, 109, 112, 115, 116, 119, 125, 127, 130, 131, 135, 136, 139, 142, 144, 145, 149, 153, 154, 155, 157, 160, 164, 165, 167, 168, 169, 171, 176, 177, 180, 181, 183, 185, 186, 187, 188, 189, 190, 192, 194, 197, 200, 201, 202, 206, 207, 210, 211, 212, 214, 215, 216, 219, 220, 221, 222, 223, 224, 226, 227, 228, 229, 230, 231, 232, 233, 234, 238, 239, 240, 242, 243, 245, 253, 254, 256, 257, 258, 260, 261, 262, 267, 268, 270, 27

In [13]:
final_fitnesses = np.array([fitness(ind) for ind in population])
best_idx = np.argmax(final_fitnesses)
best_individual = population[best_idx]
selected_features = [i for i in range(N_FEATURES) if best_individual[i] == 1]

print("\n✅ Best Accuracy:", final_fitnesses[best_idx])
print("✅ Selected Feature Indices:", selected_features)
print("✅ Total Features Selected:", len(selected_features))

# Save selected features (if X had column names)
pd.Series(selected_features).to_csv("/Users/lubhanamutha/Desktop/Cancer-Prediction-Project/trial2/binary_ga_selected_features.csv", index=False)
print("Selected features saved to ga_selected_features.csv")



✅ Best Accuracy: [0.98820905]
✅ Selected Feature Indices: [6, 8, 10, 11, 12, 14, 16, 20, 25, 26, 27, 29, 31, 33, 36, 37, 38, 39, 42, 43, 45, 47, 48, 50, 52, 54, 56, 60, 62, 63, 64, 66, 67, 68, 70, 71, 72, 73, 74, 75, 76, 77, 79, 82, 85, 86, 87, 88, 89, 91, 92, 93, 94, 96, 98, 99, 100, 103, 105, 107, 109, 112, 115, 116, 119, 125, 127, 130, 131, 135, 136, 139, 142, 144, 145, 149, 153, 154, 155, 157, 160, 164, 165, 167, 168, 169, 171, 176, 177, 180, 181, 183, 185, 186, 187, 188, 189, 190, 192, 194, 197, 200, 201, 202, 206, 207, 210, 211, 212, 214, 215, 216, 219, 220, 221, 222, 223, 224, 226, 227, 228, 229, 230, 231, 232, 233, 234, 238, 239, 240, 242, 243, 245, 253, 254, 256, 257, 258, 260, 261, 262, 267, 268, 270, 271, 272, 273, 275, 276, 277, 278, 285, 287, 288, 289, 290, 296, 302, 304, 305, 306, 308, 309, 310, 311, 312, 314, 318, 319, 320, 323, 326, 328, 331, 332, 333, 335, 338, 339, 340, 341, 342, 343, 348, 349, 350, 352, 353, 354, 355, 359, 361, 362, 363, 364, 365, 369, 371, 372, 377

In [14]:
selected_features_mask = best_individual.astype(bool)

# 2. Get the column names of the selected features
selected_gene_names = X.columns[selected_features_mask]

# 3. Create the new, reduced DataFrame of features
X_selected = X[selected_gene_names]

# 4. Combine the selected features with the original 'y' labels
# axis=1 adds the 'y' Series as a new column
final_dataset = pd.concat([X_selected, y], axis=1)

# 5. Save the final dataset to a new CSV
file_path = "/Users/lubhanamutha/Desktop/Cancer-Prediction-Project/trial2/binary_ga_selected_features_and_labels.csv"
final_dataset.to_csv(file_path, index=False)

print(f"Successfully saved {X_selected.shape[1]} features and {len(final_dataset)} samples.")
print(f"Dataset saved to: {file_path}")

Successfully saved 9054 features and 10459 samples.
Dataset saved to: /Users/lubhanamutha/Desktop/Cancer-Prediction-Project/trial2/binary_ga_selected_features_and_labels.csv
